In [49]:
import json

In [58]:
file_path = "../data/singapore.json"
file_path = "../data/delhi.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [52]:
data.keys()

dict_keys(['success', 'epoch_generated', 'date_generated', 'slug', 'cityName', 'latitude', 'longitude', 'tags', 'noDecimalLessAllUsersPaths', 'oneDecimalLessAllUsersPaths', 'highZoomUsersPaths', 'homePrices', 'rentPrices', 'cafes', 'coworkings', 'colors', 'hipsterCenter', 'preHipsterCenter', 'neighborhoodsGeoJSONAvailable', 'neighborhoodsGeoJSONURL'])

In [62]:
cat = [x["category"] for x in data["oneDecimalLessAllUsersPaths"]]
# tags = [x["tag"] for x in data["tags"]]
len(cat)

93696

In [42]:
from datetime import datetime
datetime.fromtimestamp(1757710164)

datetime.datetime(2025, 9, 12, 20, 49, 24)

In [6]:
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class X(metaclass=Singleton):
    def __init__(self, value):
        self.value = value
        print("Hello")

    # def __new__(cls, *args, **kwargs):
    #     print("You")
    #     if cls.instance is None:
    #         cls.instance = super().__new__(cls)
    #     return cls.instance

X(1).value

Hello


1

In [8]:
from pydantic import BaseModel

class Y(BaseModel):
    s : str

Y(s="ff").model_dump_json()

'{"s":"ff"}'

In [2]:
from typing import List, Dict, Union, Any, Type

# SchemaType = Dict[str, Union[Type[str], Type[int], Type[float]]]
STRING_TO_FIELD = {
    'str'   : str,
    'int'   : int,
    'float' : float,
}

def validate_objects(schema: Dict[str, str], objects: List[Dict[str, Any]]) -> List[bool]:
    """
    Validate a list of objects against a schema.
    
    :param schema: Dict with field_name -> expected_type (str, int, float).
    :param objects: List of objects (dicts) to validate.
    :return: List of booleans, one per object (True if valid, False otherwise).
    """
    results = []
    for obj in objects:
        valid = True
        for field, expected_type_str in schema.items():
            expected_type = STRING_TO_FIELD[expected_type_str]
            if field not in obj:
                valid = False
                break
            value = obj[field]
            if not isinstance(value, expected_type):
                # Allow int where float is expected
                if expected_type is float and isinstance(value, int):
                    continue
                valid = False
                break
        results.append(valid)
    return results

schema = {
    "name": "str",
    "age": "int",
    "salary": "float"
}

data = [
    {"name": "Alice", "age": 30, "salary": 50000.0},
    {"name": "Bob", "age": "not an int", "salary": 45000.0},
    {"name": "Charlie", "age": 25, "salary": 42000}  # int accepted for float
]

print(validate_objects(schema, data))
# ➝ [True, False, True]


[True, False, True]


In [4]:
from typing import Dict, List, Optional, Union, Literal
from pydantic import BaseModel


# ------------------------
# B-Tree for numeric indexes
# ------------------------
class BTreeNode(BaseModel):
    key: int
    doc_ids: Optional[List[str]] = None
    left: Optional["BTreeNode"] = None
    right: Optional["BTreeNode"] = None


BTreeNode.model_rebuild()


# ------------------------
# Two index variants
# ------------------------
class StringIndex(BaseModel):
    index_type: Literal["string"]
    mapping: Dict[str, List[str]]  # value -> list of blob_ids


class NumericIndex(BaseModel):
    index_type: Literal["numeric"]
    root: BTreeNode


# ------------------------
# Unified Index
# ------------------------
Index = Union[StringIndex, NumericIndex]

json_data = {
    "index_type": "numeric",
    "root": {
        "key": 25,
        "doc_ids": ["blob_106", "blob_107"],
        "left": {
            "key": 18,
            "doc_ids": ["blob_105"]
        },
        "right": {
            "key": 40,
            "doc_ids": ["blob_108"],
            "left": {
                "key": 30,
                "doc_ids": ["blob_109"]
            }
        }
    }
}
# idx = Index.parse_obj(json_data)
# print(idx)

idx = NumericIndex(**json_data)
print(idx.root.right.left.key)   # ➝ 30


idx = NumericIndex(**json_data)  # or Index.parse_obj(json_data)
print(idx.root.doc_ids)
# ➝ ['blob_106', 'blob_107']

json_data = {
    "index_type": "string",
    
    "mapping": {
        "alice@example.com": ["blob_101"],
        "bob@example.com": ["blob_102", "blob_110"]
    }
}

idx = StringIndex(**json_data)   # or Index.parse_obj(json_data)
print(idx)
# ➝ index_type='string' mapping={'alice@example.com': ['blob_101'], ...}


30
['blob_106', 'blob_107']
index_type='string' mapping={'alice@example.com': ['blob_101'], 'bob@example.com': ['blob_102', 'blob_110']}


In [9]:
x = {'a': 5, 'b': 10, 'c': 15}
y = {**x, 'a': 20}
z = [1,2,3,"ff"]
obj = str(z).encode()
obj.decode('utf-8')

"[1, 2, 3, 'ff']"

In [3]:
! pip list | grep walrus
! from walrusdb.types import Object

walrus-python      0.1.0
walrusdb           0.0.1       C:\Users\nishk\Documents\Cudding\btchd\walrusdb



[notice] A new release of pip available: 22.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
'from' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
a = [1,2,3,4]
a.remove(3)
a

[1, 2, 4]

In [2]:
import time
import asyncio

async def timer(sec):
    await time.sleep(sec)
    return 5

In [5]:
import argparse

class DefaultNamespace(argparse.Namespace):
    def __getattr__(self, name):
        # Return None if the attribute doesn't exist
        return None


# Example usage
args = DefaultNamespace(json_path="data.json", output="results.json")

In [6]:
print(args.json_path)  # -> "data.json"
print(args.output)     # -> "results.json"
print(args.api_key)    # -> None (not set)
print(args.update)  

data.json
results.json
None
None
